Importing Libraries and dataset

In [1]:
import pandas as pd 
import numpy as np 
import sqlite3 as sq
from bs4 import BeautifulSoup

In [3]:
conn = sq.connect("database.sqlite")
data = pd.read_sql_query('SELECT * FROM REVIEWS WHERE SCORE!=3 ',conn)  #assuming 3 as neutral reviews and hence removing the same from our datastet
print(data.head())
print(data.shape)

   Id   ProductId          UserId                      ProfileName   
0   1  B001E4KFG0  A3SGXH7AUHU8GW                       delmartian  \
1   2  B00813GRG4  A1D87F6ZCVE5NK                           dll pa   
2   3  B000LQOCH0   ABXLMWJIXXAIN  Natalia Corres "Natalia Corres"   
3   4  B000UA0QIQ  A395BORC6FGVXV                             Karl   
4   5  B006K2ZZ7K  A1UQRSCLF8GW1T    Michael D. Bigham "M. Wassir"   

   HelpfulnessNumerator  HelpfulnessDenominator  Score        Time   
0                     1                       1      5  1303862400  \
1                     0                       0      1  1346976000   
2                     1                       1      4  1219017600   
3                     3                       3      2  1307923200   
4                     0                       0      5  1350777600   

                 Summary                                               Text  
0  Good Quality Dog Food  I have bought several of the Vitality canned d...  
1 

In [4]:
#count of rows having neutral rows
print("original_data_shape =",pd.read_sql_query('SELECT COUNT(*) FROM REVIEWS',conn))
print("neutral_reviews_count = ",pd.read_sql_query('Select count(*) from REVIEWS WHERE SCORE=3',conn))

#checking for redundancy where HelpFulnessNumerator is greater than HelpfulnessDenominator
print(pd.read_sql_query('select  * from REVIEWS WHERE HelpFulnessNumerator>HelpFulnessDenominator',conn).shape)  
conn.close()         #closing DB connection

original_data_shape =    COUNT(*)
0    568454
neutral_reviews_count =     count(*)
0     42640
(2, 10)


In [5]:
# score of > 3 is positive and <3 taken as negative
def re_score(x):
    if x < 3: 
        return 0
    return 1

data['Score'] = data['Score'].map(re_score)
data.head() 

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
#kind = type of sorting (read np.sort)
#position of null positions (na_postion) = first or last
sorted_data = data.sort_values('ProductId',axis='index',ascending=True,kind='mergesort',na_position='last')

In [7]:
df = sorted_data.drop_duplicates(subset={"ProductId","UserId","Time","ProfileName","Text"},keep='first')
df.shape

(524691, 10)

In [8]:
df = df[df['HelpfulnessNumerator']<=df['HelpfulnessDenominator']]  #two records were present
df.shape

(524689, 10)

In [9]:
df.head(5)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138676,150493,0006641040,AMX0PJKV4PPNJ,"E. R. Bird ""Ramseelbird""",71,72,1,1096416000,Read it once. Read it twice. Reading Chicken S...,"These days, when a person says, ""chicken soup""..."
138677,150494,0006641040,AYZ0PR5QZROD1,Mother of 3 girls,3,3,1,1173312000,Family favorite,All of my children love this book. My first g...
138678,150496,0006641040,A3KKR87BJ0C595,"Gretchen Goodfellow ""Lover of children's lit""",3,3,1,1111363200,"You'll use it once, you'll use it twice",One of my earliest memories is of this book. ...
138679,150497,0006641040,A1HKYQOFC8ZZCH,"Maria Apolloni ""lanarossa""",2,2,0,1334707200,"The story is great, the softcover book is disa...",I give five stars to the Maurice Sendak story....
138680,150498,0006641040,A3SJWISOCP31TR,R. J. Wells,2,2,1,1176336000,A Gem of a Book,This is a wonderful little book. I loved it 40...


In [10]:
# checking for redundant values
print(df.isna().any())

Id                        False
ProductId                 False
UserId                    False
ProfileName               False
HelpfulnessNumerator      False
HelpfulnessDenominator    False
Score                     False
Time                      False
Summary                   False
Text                      False
dtype: bool


Text Preprocessing

In [11]:
import re

In [12]:
text1 = df['Text'].values[0]
text1

'These days, when a person says, "chicken soup" they\'re probably going to follow up those words with, "for the soul" or maybe "for the teenaged soul".  Didn\'t used to be that way.  Why I can remember a time when if a person said, "chicken soup" those words were followed by an enthusiastic "with rice!".  Such was the power of Maurice Sendak\'s catchy 1962 children\'s book.  I am pleased to report that if you care to read this book again today, you will find it hasn\'t dimished a jot in terms of frolicksome fun.  In this book we are led through a whirlwind chicken soup year with our host, a boy who bears no little resemblance to Sendak\'s other great rhyming tale "Pierre" (in looks if not demeanor).  It\'s a catchy flouncy bouncy combo of soup and the people who love it so.<br /><br />This is ostensibly a book meant to teach your children the different months of the year.  Each month gets its own rhythmic poem and accompanying illustration.  These are fairly simple pen and ink drawings

In [13]:
#removing URLs from text
text0  = "www.pornhub is my favourite web"
text0 = re.sub(r'http\S+','',text0)
text0 = re.sub(r'www\S+','',text0)
text0

' is my favourite web'

In [14]:
#removing HTML tags
from bs4 import BeautifulSoup
text2 = "<html><body><p>We have used the Victor fly bait for 3 seasons.  Can't beat it.  Great product!</p></body></html>"
text2 = BeautifulSoup(text2).get_text()
text2


"We have used the Victor fly bait for 3 seasons.  Can't beat it.  Great product!"

In [15]:

#Removing special character and punctuations
# text1 = '''These days, when a person says, "chicken soup" they\'re 
# probably going to follow up those words with, "for the soul" or maybe "for the teenaged soul".  
# Didn\'t used to be that way.  Why I can remember a time when if a person said, "chicken soup" 
# those words were followed by an enthusiastic "with rice!".  Such was the power of Maurice Sendak\'s catchy 
# 1962 children\'s book.  I am pleased to report that if you care to read this book again today, 
# you will find it hasn\'t dimished a jot in terms of frolicksome fun.  
# In this book we are led through a whirlwind chicken soup year with our host, 
# a boy who bears no little resemblance to Sendak\'s other great rhyming tale "Pierre" (in looks if not demeanor).  
# It\'s a catchy flouncy bouncy combo of soup and the people who love it so.<br /><br />This is ostensibly a book 
# meant to teach your children the different months of the year.  Each month gets its own rhythmic poem and accompanying illustration.  
# These are fairly simple pen and ink drawings with the occasional splash of blue (in varying shades), yellow, gray, and green.  
# You may wonder how an author could ever hope to come up with twelve highly original soup-related poems.  I mean, honestly, how much 
# is there to say about even the fanciest soup, let alone chicken soup with rice?  Quite a lot, as it happens.  In the cold winter
#  months soup is supped while sliding on ice, while celebrating the birthday of a snowman, and in a gusty gale as a whale. 
#    In the spring there\'s robin\'s nest soup, soup to cure drooping roses, and soup stolen by jealous March winds.  
#    Our hero postulates the potential joys that could come of being a cooking pot, stewing soup or (oddly enough) as 
#    "a baubled bangled Christmas tree".<br /><br />Not to degrade the reading skills of parents everywhere,
#      but I cannot recommend enough getting an audio version of this tale to accompany your child\'s reading. 
#        Though I am now a wise and cultured 26 year-old (the years have been kind to me in this, my old age)
#          I can still remember the chicken soup with rice tune.  Heck, I read this entire book recently and 
#          found I could do the song perfectly with each and every line.  Now maybe you have your own particular chicken soup with rice song style that
#            you\'re just loathe to give up.  If so, fine.  I understand why you might not want to taint your already existing chicken soup melody.
#                But if you haven\'t found a jingle to accompany this book, get the audio version immediately,
#                  if not sooner.  Until you can sing "Whoopy once, whoopy twice, whoopy chicken soup with rice" with the correct oomph,
#                    you\'re missing out.<br /><br />I take my "Chicken Soup With Rice" readings seriously. 
#                      This book was the "Chicka Chicka Boom Boom" of its day, and still remains the catchiest method to teach kids the months of the year.  
#                      It is also seriously in danger of being forgotten.  So pull out your old accordion and strap on your dancing shoes.  
#                      The time for yukkin\' it up to a merry dance of poultry broth is here.  It\'s Sendak at his finest.'''

# text1 = re.sub('[^A-Za-z0-9]+',' ',text1)
# text1

In [16]:
#removing alpha-numeric characters
# text2 = re.sub('\S*\d\S*','',text2)
# text2

In [17]:
#performing decontraction
def decontraction(phrase):
    phrase = re.sub(r"won't","will not",phrase)
    phrase = re.sub(r"can't","can not",phrase)
    phrase = re.sub(r"n't","not",phrase)
    phrase = re.sub(r"\'re","are",phrase)
    phrase = re.sub(r"\'s"," is",phrase)
    phrase = re.sub(r"\'d"," would",phrase)
    phrase = re.sub(r"\'ll"," will",phrase)
    phrase = re.sub(r"\'t"," not",phrase)
    phrase = re.sub(r"\'ve"," have",phrase)
    phrase = re.sub(r"\'m"," am",phrase)
    return phrase


In [18]:
# text4 = 'Mrs. May\'s pumpkin crunch is a great snack. For those of you watching calories and intake, be aware that the 2oz portions are two servings. Most of Mrs. May\'s products are great, but with the fat at 11g per serving, i find it\'s not a complete guilt free snack. However, the pumpkin seed crunch has 9gms of protein per serving vs. the usual 4gms in their other snacks. So if you\'re monitoring your intake and want to indulge in these now and then, the pumpkin seed crunch is the healthiest option of the bunch.<br /><br />That said, they are all tasty and certainly not bad for you, just a little high in fat if you\'re watching your weight.'
# print(decontraction(text4))

In [19]:
# # converting texts into lower case
# def lower_case(phrase):
#     phrase = ''.join(i.lower() for i in phrase)
#     return phrase

# print(lower_case(text4))

In [20]:
#removing stopwords

import nltk 
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
def remove_stop_words(phrase):
    phrase  = " ".join(i for i in phrase.split() if i.lower() not in stopwords)
    return phrase

# print(remove_stop_words(text4))

In [21]:
#stemming words
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')


In [23]:
#single block to perform all text-preprocessing
from tqdm import tqdm

corpus = []
for review in tqdm(df['Text'].values) :
    review = re.sub(r'http\S+','',review)
    review = re.sub(r'www\S+','',review)
    review = re.sub('[^A-Za-z0-9]+'," ",review)
    review = BeautifulSoup(review).get_text()
    review = re.sub('\S*\d\S*',' ',review)
    review = ' '.join(i.lower() for i in review.split() if i.lower() not in stopwords)
    review = decontraction(review)
    review = ' '.join(stemmer.stem(i) for i in review.split())
    corpus.append(review)



100%|██████████| 524689/524689 [06:44<00:00, 1297.69it/s]


In [153]:
# corpus = []
# for review in tqdm(df['Text'].values) :
#     review = re.sub(r'http\S+','',review)
#     review = re.sub(r'www\S+','',review)
#     review = re.sub('[^A-Za-z0-9]+'," ",review)
#     review = BeautifulSoup(review).get_text()
#     review = re.sub('\S*\d\S*',' ',review)
#     review = ' '.join(i.lower() for i in review.split() if i.lower() not in stopwords)
#     review = decontraction(review)
#     # review = ' '.join(stemmer.stem(i) for i in review.split())
#     corpus.append(review)


100%|██████████| 524689/524689 [02:36<00:00, 3351.56it/s]


In [24]:
corpus[999]

'whole idea keep flea hous invad tri everi thing spray power name temporari solut best best respons invas trap invad trap flea remov trap kid yes trap work remov flea dead chemic flea get hous prefer kill resid hous tri trap br br mike'

Vectorization

1. Bag of Words

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

def bag_of_words(data):
    bow = CountVectorizer()
    return bow.fit(data)
    

2. TF-IDF

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tf_idf(data):
    cv= TfidfVectorizer()
    return cv.fit(data)


#https://datascience.stackexchange.com/questions/12321/difference-between-fit-and-fit-transform-in-scikit-learn-models

3.  Word2vec

In [27]:
from gensim.models import Word2Vec
#for word2vec , data is required as list of lists
#more the data, better is the performance
# http://kavita-ganesan.com/gensim-word2vec-tutorial-starter-code/#.XR0cft9fiXJ


ds = []
for e in corpus:
    ds.append(e.split(' '))

model = Word2Vec(ds,vector_size=150,window=25,min_count=2)    
# size is number of dimensions of vector that you want to build
# window is the maximum distance between two similar words
# min_count is the minimum number of times the word must appear in corpus
# workers is the numbers of cpu that you want to use

In [28]:
type(model)

gensim.models.word2vec.Word2Vec

In [31]:
w1 = 'hot'
print(model.wv.most_similar(w1))

[('heat', 0.5280380249023438), ('warm', 0.5267618894577026), ('spicey', 0.4505113661289215), ('hotter', 0.44809794425964355), ('tobasco', 0.4477368891239166), ('cold', 0.4477030038833618), ('hottest', 0.4324588477611542), ('spiciest', 0.4288228452205658), ('kesar', 0.4284977316856384), ('hotsauc', 0.41397127509117126)]


In [33]:
print(model.wv['tasti'])

[ 1.321804   -1.4299887  -0.50470334  2.822866   -1.9297407   2.524284
  0.5868802  -2.608637    1.0385221   2.3997345   1.7594752   0.9020148
  2.1739986   1.3392415   1.0462416  -1.0565667  -0.12358692  2.2723043
  2.3906474   0.59698945  0.8223713   0.8823528  -2.2994554   3.4944494
 -0.9782466  -0.99983686  0.38015124 -0.6163857   1.3811598  -0.7938637
 -1.5439202  -0.9936711   2.4911823  -0.38046962 -0.8290098   0.9464108
  0.61268705 -1.221799   -5.237058    1.5755296  -0.66747874 -0.31991908
 -0.8811557   1.5913005  -0.8251669  -0.5456122   0.76949716 -2.8592975
 -2.673991    1.974981   -0.17506124  1.524496    2.4258711  -4.517895
 -1.2311157   1.8715397  -0.17031655  1.4157583   0.11759083  0.63736933
  1.2069964  -1.0731347   2.6334758   1.9463329   0.3597584  -2.2332835
 -0.6696185  -2.8144088   0.42790574  0.24060905 -1.6640558   0.94563293
 -1.4106219   5.1571007  -2.7481146  -1.484586   -0.97233564 -3.178281
 -2.3011608   0.9279654   1.0441742  -2.6237488  -1.6940354  -2.

Applying Logistic Regression

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split


In [35]:
preprocessed_df = df.copy()
preprocessed_df['Text'] = corpus
print(preprocessed_df.shape)
print(preprocessed_df['Score'].value_counts())


(524689, 10)
Score
1    442966
0     81723
Name: count, dtype: int64


In [36]:
preprocessed_df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138676,150493,0006641040,AMX0PJKV4PPNJ,"E. R. Bird ""Ramseelbird""",71,72,1,1096416000,Read it once. Read it twice. Reading Chicken S...,day person say chicken soup probabl go follow ...
138677,150494,0006641040,AYZ0PR5QZROD1,Mother of 3 girls,3,3,1,1173312000,Family favorite,children love book first grader got christma l...
138678,150496,0006641040,A3KKR87BJ0C595,"Gretchen Goodfellow ""Lover of children's lit""",3,3,1,1111363200,"You'll use it once, you'll use it twice",one earliest memori book mother read us consta...
138679,150497,0006641040,A1HKYQOFC8ZZCH,"Maria Apolloni ""lanarossa""",2,2,0,1334707200,"The story is great, the softcover book is disa...",give five star mauric sendak stori one star pr...
138680,150498,0006641040,A3SJWISOCP31TR,R. J. Wells,2,2,1,1176336000,A Gem of a Book,wonder littl book love year ago twin love enjo...


In [5]:
train_data , test_data = train_test_split(preprocessed_df,random_state=20,test_size=0.25)

NameError: name 'train_test_split' is not defined

Logistic Regression on BOW

In [4]:
bow = bag_of_words(train_data['Text'])
train_data_bow = bow.transform(train_data['Text'])
test_data_bow = bow.transform(test_data['Text'])

NameError: name 'bag_of_words' is not defined

In [3]:
s = ['whole idea keep flea hous invad tri everi thing spray power name temporari solut best best respons invas trap invad trap flea remov trap kid yes trap work remov flea dead chemic flea get hous prefer kill resid hous tri trap br br mike']
x = bow.transform(s)

NameError: name 'bow' is not defined

In [2]:
y = x.toarray()

NameError: name 'x' is not defined

In [1]:
print(train_data_bow.shape, train_data['Score'].shape)
print(test_data_bow.shape)
print(train_data['Score'].value_counts())

NameError: name 'train_data_bow' is not defined

In [84]:
#Normalizing data
from sklearn.preprocessing import Normalizer
train_data_bow_n = Normalizer().fit_transform(train_data_bow)
test_data_bow_n  = Normalizer().fit_transform(test_data_bow)

In [85]:
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(train_data_bow,train_data['Score'])

LogisticRegression(max_iter=1000)

In [89]:
y_pred = model_lr.predict(test_data_bow)
score = accuracy_score(y_pred,test_data['Score'])*100
print("Accuracy acheived by Logistic Regression using Bag of Words =",score,"%")

Accuracy acheived by Logistic Regression using Bag of Words = 93.35076578259246 %


Logistic Regression on TF-IDF

In [93]:
tfidf = tf_idf(train_data['Text'])
train_data_tfidf = tfidf.transform(train_data['Text'])
test_data_tfidf = tfidf.transform(test_data['Text'])
print(train_data['Text'].shape)
print(test_data['Text'].shape)
print(train_data_tfidf.shape)
print(test_data_tfidf.shape)

(393516,)
(131173,)
(393516, 63423)
(131173, 63423)


In [100]:
train_data_tfidf_n = Normalizer().fit_transform(train_data_tfidf)
test_data_tfidf_n = Normalizer().fit_transform(test_data_tfidf)

In [101]:
model_lr_tfidf = LogisticRegression(max_iter=1000)
model_lr_tfidf.fit(train_data_tfidf_n,train_data['Score'])

LogisticRegression(max_iter=1000)

In [102]:
y_pred = model_lr_tfidf.predict(test_data_tfidf_n)
score = accuracy_score(y_pred,test_data['Score'])*100
print("Accuracy acheived by Logistic Regression using TFIDF =",score,"%")

Accuracy acheived by Logistic Regression using TFIDF = 92.99169798662834 %


Logistic Regression on Word2vec

A. Average word2vec

In [105]:
def avg_w2v(data):
    avg_list = []
    for i in tqdm(data):
        vec = np.zeros(150)
        count = 0
        for j in i.split(' '):
            try:
                vec+= model.wv[j]
                count+=1
            except: 
                pass
        if count!=0:
            vec = vec/count
            avg_list.append(vec)
        else:
            avg_list.append(np.zeros(150))

    print("Total number of the vector : ",len(avg_list))  #Length of the list
    print("Dimension of Vector : ",len(avg_list[1])) # Length of average vector
    return np.array(avg_list)   

In [137]:
def avg_w2v(data,model):
    avg_list = []
    for i in tqdm(data):
        vec = np.zeros(150)
        count = 0
        for j in i.split(' '):
            try:
                vec+= model[j]
                count+=1
            except: 
                pass
        if count!=0:
            vec = vec/count
            avg_list.append(vec)
        else:
            avg_list.append(np.zeros(150))

    print("Total number of the vector : ",len(avg_list))  #Length of the list
    print("Dimension of Vector : ",len(avg_list[1])) # Length of average vector
    return np.array(avg_list)  

In [106]:
# train_data_w2v = avg_w2v(train_data['Text'])
# test_data_w2v = avg_w2v(test_data['Text'])

# train_data_w2v_n = Normalizer().fit_transform(train_data_w2v)
# test_data_w2v_n = Normalizer().fit_transform(test_data_w2v)

100%|██████████| 393516/393516 [00:56<00:00, 6989.52it/s] 


Total number of the vector :  393516
Dimension of Vector :  150


100%|██████████| 131173/131173 [00:14<00:00, 9117.29it/s]


Total number of the vector :  131173
Dimension of Vector :  150


In [138]:
train_data_w2v = avg_w2v(train_data['Text'],model.wv)
test_data_w2v = avg_w2v(test_data['Text'],model.wv)

train_data_w2v_n = Normalizer().fit_transform(train_data_w2v)
test_data_w2v_n = Normalizer().fit_transform(test_data_w2v)

100%|██████████| 393516/393516 [00:46<00:00, 8377.18it/s]


Total number of the vector :  393516
Dimension of Vector :  150


100%|██████████| 131173/131173 [00:13<00:00, 9708.52it/s] 


Total number of the vector :  131173
Dimension of Vector :  150


In [146]:
model_lr_w2v = LogisticRegression(max_iter=400)
model_lr_w2v.fit(train_data_w2v_n,train_data['Score'])


LogisticRegression(max_iter=400)

In [145]:
y_pred = model_lr_w2v.predict(test_data_w2v)
score = accuracy_score(y_pred,test_data['Score'])*100
print("Accuracy acheived by Logistic Regression using avg w2v =",score,"%")


Accuracy acheived by Logistic Regression using avg w2v = 89.66784322993298 %


Using google dataset for word2vec

In [114]:
import gensim
import gensim.downloader as api
print(list(gensim.downloader.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [116]:
# wv = api.load('word2vec-google-news-300')
# wv.save('../Google news dataset/vectors.kv')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [134]:
wv.similarity('good','sexy')


0.24171118

In [136]:
wv['tasty']

array([-0.17382812,  0.02404785, -0.03015137,  0.27734375, -0.09228516,
        0.0144043 ,  0.11376953,  0.12597656, -0.19433594,  0.30859375,
       -0.18652344, -0.17578125,  0.14941406, -0.02819824, -0.06982422,
        0.30273438,  0.01989746, -0.05224609,  0.01361084,  0.06542969,
        0.06689453,  0.17480469,  0.22070312,  0.13964844, -0.04833984,
       -0.30859375, -0.14746094,  0.18066406,  0.02746582, -0.00228882,
       -0.05859375,  0.07763672,  0.33789062,  0.07568359, -0.25      ,
       -0.01806641,  0.21972656, -0.16796875,  0.11621094,  0.20703125,
        0.15625   , -0.23828125,  0.08544922,  0.10986328,  0.00170135,
       -0.33203125, -0.05834961,  0.43359375,  0.05810547,  0.16210938,
       -0.09277344,  0.13378906,  0.02404785,  0.04321289, -0.13378906,
        0.05664062,  0.20605469, -0.24707031, -0.06054688, -0.15820312,
       -0.15820312,  0.06884766, -0.23535156, -0.00361633, -0.02941895,
       -0.26171875, -0.10009766, -0.42578125,  0.00531006,  0.05

In [156]:
train_data_w2v1 = avg_w2v(train_data['Text'],wv)
test_data_w2v1 = avg_w2v(test_data['Text'],wv)

train_data_w2v_n = Normalizer().fit_transform(train_data_w2v1)
test_data_w2v_n = Normalizer().fit_transform(test_data_w2v1)

100%|██████████| 393516/393516 [01:21<00:00, 4825.71it/s]


Total number of the vector :  393516
Dimension of Vector :  150


100%|██████████| 131173/131173 [00:24<00:00, 5430.77it/s]


Total number of the vector :  131173
Dimension of Vector :  150


In [157]:
model_lr_w2v = LogisticRegression(max_iter=400)
model_lr_w2v.fit(train_data_w2v_n,train_data['Score'])

LogisticRegression(max_iter=400)

In [158]:
y_pred = model_lr_w2v.predict(test_data_w2v)
score = accuracy_score(y_pred,test_data['Score'])*100
print("Accuracy acheived by Logistic Regression using avg w2v trained on google dataset =",score,"%")

Accuracy acheived by Logistic Regression using avg w2v trained on google dataset = 84.44268256424722 %
